## Creando base de datos para entrenamiento

In [1]:
import sqlite3
from faker import Faker
import random

# Initialize Faker
fake = Faker()

# Connect to (or create) database
conn = sqlite3.connect("customers.db")
cursor = conn.cursor()

# Create table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    email TEXT UNIQUE,
    phone TEXT UNIQUE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    status TEXT CHECK(status IN ('active', 'cancelled', 'prospect')) DEFAULT 'active'
)
""")

# Generate 25 fake customers
statuses = ["active", "cancelled", "prospect"]
customers = []
for _ in range(25):
    name = fake.name()
    email = fake.unique.email()
    phone = fake.unique.msisdn()[:9]  # shorter phone format
    status = random.choice(statuses)
    customers.append((name, email, phone, status))

# Insert customers
cursor.executemany("INSERT OR IGNORE INTO customers (name, email, phone, status) VALUES (?, ?, ?, ?)", customers)

# Commit and close
conn.commit()
conn.close()

customers[:5]  # show a sample of the generated customers


[('Lori Scott', 'nunezdavid@example.net', '557934276', 'cancelled'),
 ('Luis Medina', 'cantrellamber@example.net', '709433099', 'active'),
 ('Kelsey Hart', 'charleneperez@example.com', '822967025', 'prospect'),
 ('Ashley Smith', 'robertlloyd@example.net', '219296401', 'cancelled'),
 ('Jessica Gillespie', 'salinaskyle@example.net', '080627170', 'cancelled')]

In [10]:
def row_to_customer_dict(row):
    return {
        "customer_id": row[0],
        "name": row[1],
        "email": row[2],
        "phone": row[3],
        "status": row[4]
    }

In [3]:
def check_customer(identifier: str):
    conn = sqlite3.connect("customers.db")
    cursor = conn.cursor()

    cursor.execute(
        """
        SELECT customer_id, name, email, phone, status 
        FROM customers 
        WHERE email = ? 
                   OR phone = ? 
                   OR name LIKE ?
    """,
        (identifier, identifier, f"%{identifier}%"),
    )

    result = cursor.fetchone()
    conn.close()

    if result:
        return row_to_customer_dict(result)
    else:
        return None

In [52]:
print(check_customer("alice@email.com"))

print(check_customer("unknown@email.com"))


{'customer_id': 1, 'name': 'Alice Lopez', 'email': 'alice@email.com', 'phone': '555-123-456', 'status': 'active'}
None


In [47]:
def get_all_customers():
    # Retrieve customers
    conn = sqlite3.connect("customers.db")
    cursor = conn.cursor()

    cursor.execute("""
        SELECT customer_id, name, email, phone, status
        FROM customers
    """)

    rows = cursor.fetchall()
    conn.close()
    customers = list(map(row_to_customer_dict, rows))
    return customers
customers=get_all_customers()
customers[:5]

[{'customer_id': 1,
  'name': 'Alice Lopez',
  'email': 'alice@email.com',
  'phone': '555-123-456',
  'status': 'active'},
 {'customer_id': 2,
  'name': 'Bob Perez',
  'email': 'bob@email.com',
  'phone': '555-999-888',
  'status': 'cancelled'},
 {'customer_id': 3,
  'name': 'Carla Ruiz',
  'email': None,
  'phone': '555-777-333',
  'status': 'active'},
 {'customer_id': 7,
  'name': 'Brandon Lopez',
  'email': 'brentwright@example.net',
  'phone': '515132110',
  'status': 'active'},
 {'customer_id': 8,
  'name': 'Jeffrey Berry',
  'email': 'sfuller@example.com',
  'phone': '758005699',
  'status': 'prospect'}]

In [ ]:
import json
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()

customers = get_all_customers()
print(customers[:5])

# Example message pools
customer_msgs = [
    "Hi, I want to know your internet plans.",
    "My service is down since this morning.",
    "Can I upgrade my package?",
    "I need help with my bill.",
    "Do you have fiber in my area?",
    "I’m interested in becoming a customer.",
    "How much is the premium plan?",
    "Can I cancel anytime?",
]

bot_msgs = [
    "Sure! We offer Basic, Plus, and Premium.",
    "I see, let me check that for you.",
    "I’m escalating your issue to a support specialist.",
    "Yes, you can upgrade anytime.",
    "Please provide your account number.",
    "The premium plan costs $80/month.",
    "Yes, you can cancel anytime.",
]

agent_msgs = [
    "Hello, I’ll take over from here.",
    "Can you confirm your address?",
    "I’m creating a ticket for your issue.",
    "We’ll send a technician tomorrow.",
]

# Generate 10 fake conversations with identification first
conversations = []
for i in range(10):
    cust = random.choice(customers)
    conv_id = f"c{i+1}"
    start_time = datetime.now() - timedelta(days=random.randint(0, 30), hours=random.randint(0,12))
    current_time = start_time

    messages = []

    # First step: identification by customer
    messages.append({
        "sender": "bot",
        "text": "Hello! Can you provide your email or phone so I can identify you?",
        "timestamp": current_time.isoformat()
    })
    current_time += timedelta(seconds=5)

    messages.append({
        "sender": "customer",
        "text": cust["email"],  # email from the customer tuple
        "timestamp": current_time.isoformat()
    })
    current_time += timedelta(seconds=1)

    messages.append({
        "sender": "bot",
        "text": f"Welcome back, {cust['name']}! How can I help you today?",
        "timestamp": current_time.isoformat()
    })
    current_time += timedelta(minutes=1)

    # Generate 2-4 additional exchanges
    exchanges = random.randint(2, 4)
    for _ in range(exchanges):
        messages.append({
            "sender": "customer",
            "text": random.choice(customer_msgs),
            "timestamp": current_time.isoformat()
        })
        current_time += timedelta(minutes=random.randint(1,5))

        if random.random() < 0.8:
            reply = random.choice(bot_msgs)
            sender = "bot"
        else:
            reply = random.choice(agent_msgs)
            sender = "agent"
        messages.append({
            "sender": sender,
            "text": reply,
            "timestamp": current_time.isoformat()
        })
        current_time += timedelta(minutes=random.randint(1,5))

    conversations.append({
        "conversation_id": conv_id,
        "customer_id": cust["customer_id"],
        "messages": messages,
        "status": "closed"
    })

# Save to JSON
with open("conversations.json", "w") as f:
    json.dump(conversations, f, indent=2)

conversations[:2]  # show first 2 conversations as sample

[{'customer_id': 1, 'name': 'Alice Lopez', 'email': 'alice@email.com', 'phone': '555-123-456', 'status': 'active'}, {'customer_id': 2, 'name': 'Bob Perez', 'email': 'bob@email.com', 'phone': '555-999-888', 'status': 'cancelled'}, {'customer_id': 3, 'name': 'Carla Ruiz', 'email': None, 'phone': '555-777-333', 'status': 'active'}, {'customer_id': 7, 'name': 'Brandon Lopez', 'email': 'brentwright@example.net', 'phone': '515132110', 'status': 'active'}, {'customer_id': 8, 'name': 'Jeffrey Berry', 'email': 'sfuller@example.com', 'phone': '758005699', 'status': 'prospect'}]


[{'conversation_id': 'c1',
  'customer_id': 17,
  'messages': [{'sender': 'bot',
    'text': 'Hello! Can you provide your email or phone so I can identify you?',
    'timestamp': '2025-08-15T13:37:48.455656'},
   {'sender': 'customer',
    'text': 'megan12@example.net',
    'timestamp': '2025-08-15T13:37:53.455656'},
   {'sender': 'bot',
    'text': 'Welcome back, Michael Roy! How can I help you today?',
    'timestamp': '2025-08-15T13:37:54.455656'},
   {'sender': 'customer',
    'text': 'Can I cancel anytime?',
    'timestamp': '2025-08-15T13:38:54.455656'},
   {'sender': 'bot',
    'text': 'The premium plan costs $80/month.',
    'timestamp': '2025-08-15T13:41:54.455656'},
   {'sender': 'customer',
    'text': 'My service is down since this morning.',
    'timestamp': '2025-08-15T13:42:54.455656'},
   {'sender': 'bot',
    'text': 'I see, let me check that for you.',
    'timestamp': '2025-08-15T13:44:54.455656'}],
  'status': 'closed'},
 {'conversation_id': 'c2',
  'customer_id': 1

In [ ]:
import spacy
import re

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Regex patterns for email and phone
email_pattern = r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+"
phone_pattern = r"\b\d{7,15}\b"

def identify_customer(user_text):
    name = None
    email = None
    phone = None

    # Extract email
    email_match = re.search(email_pattern, user_text)
    email = email_match.group() if email_match else None

    # Extract phone
    phone_match = re.search(phone_pattern, user_text)
    phone = phone_match.group() if phone_match else None

    # Extract name using spaCy NER
    doc = nlp(user_text)

    for ent in doc.ents:
        # print(ent.label_, ent.text)
        if ent.label_ == "PERSON":
            name = ent.text
            break
        if ent.label_ == "DATE":
            phone = ent.text

    if phone:
        customer = check_customer(phone)
        if customer is not None:
            return customer
    if name:
        customer = check_customer(name)
        if customer is not None:
            return customer
    
    if email:
        customer = check_customer(email)
        if customer is not None:
            return customer

    return {"type": "new_client"}  # if nothing matched

# Example usage
user_texts = [
    "Hi, my name is Alice Johnson and my email is alice@example.com",
    "Hello, you can reach me at 5551-2456",
    "I am David Smith"
]

for text in user_texts:
    print(f"Input: {text}")
    print("Customer ID:", identify_customer(text))


Input: Hi, my name is Alice Johnson and my email is alice@email.com
Customer ID: {'customer_id': 1, 'name': 'Alice Lopez', 'email': 'alice@email.com', 'phone': '555-123-456', 'status': 'active'}
Input: Hello, you can reach me at 5551-2456
Customer ID: {'type': 'new_client'}
Input: I am David Smith
Customer ID: {'type': 'new_client'}
